### Peer-graded Assignment: Part 3

### Importing the libraries needed

In [142]:
import pandas as pd # library for data analsysis
import requests as request # library to handle requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


### Adding BeautifulSoup library for grabbing the text from the website

In [143]:
!pip install beautifulsoup4
!pip install lxml
from bs4 import BeautifulSoup

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: lxml in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


### Grabbing the text from the website and adding it into an array for later use

In [144]:
wikipedia_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = request.get(wikipedia_link).text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')

rows = []
for row in table.findAll('tr'):
    for cell in row.findAll('td'):
        pincode = cell.p.b.text
        for index, details in enumerate(cell.findAll('a')):
            if index == 0:
                borough = details.string
                neighborhood = ''
            elif index == 1:
                neighborhood = details.string
            else:
                neighborhood = neighborhood + ', ' + details.string
            if (index == len(cell.findAll('a')) - 1):
                if index == 0:
                    neighborhood = borough
                try: 
                    row = [pincode, borough, neighborhood]
                    rows.append(row)
                except:
                    pass

### Initializing a dataframe 'neighborhood_df' and adding the data from the rows array

In [145]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhood_df = pd.DataFrame(rows, columns=column_names)
neighborhood_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [146]:
neighborhood_df.shape

(101, 3)

In [147]:
# The code was removed by Watson Studio for sharing.

### Performed Google API call to convert address tp latitude and longitude

In [148]:
for index, row in neighborhood_df.iterrows():
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, str(str(row['Neighborhood']) + " " + str(row['Borough']) + " " + str(row['PostalCode'])).replace(" ", "%20") )
    response = request.get(url).json() # get response
    geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
    latitude = geographical_data['lat']
    longitude = geographical_data['lng']
    neighborhood_df.loc[index,'Latitude'] = latitude 
    neighborhood_df.loc[index,'Longitude'] = longitude 

In [149]:
neighborhood_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.659343,-79.365525
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722774,-79.450928
4,M7A,Queen's Park,Queen's Park,43.662843,-79.389638
5,M9A,Etobicoke,Islington Avenue,43.680233,-79.539081
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706139,-79.325415
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657658,-79.378802


### Keeping the data of the neighborhood having keyword 'Toronto'

In [150]:
toronto_df = neighborhood_df[neighborhood_df['Borough'].str.contains("Toronto")].reset_index(drop=True)

In [151]:
toronto_df.shape

(19, 5)

In [152]:
toronto_df = toronto_df.drop_duplicates(subset=['Neighborhood'], keep='first', inplace=False).reset_index(drop=True)
toronto_df.head(50)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.659343,-79.365525
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657658,-79.378802
2,M5C,Downtown Toronto,St. James Town,43.668195,-79.373771
3,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
4,M5G,Downtown Toronto,Bay Street,43.655854,-79.383778
5,M5H,Downtown Toronto,"Richmond, King",43.648918,-79.378169
6,M5J,Downtown Toronto,"Harbourfront, Union Station, Toronto Islands",43.638702,-79.385741
7,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647693,-79.380330
8,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.647906,-79.378065
9,M4R,North Toronto,North Toronto,43.715383,-79.405678


In [153]:
toronto_latitude = 43.6972285
toronto_longitude = -79.396789

In [154]:
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [155]:
# The code was removed by Watson Studio for sharing.

In [156]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = request.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [157]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Downtown Toronto
Bay Street
Richmond, King
Harbourfront, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
North Toronto
University of Toronto
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [158]:
print(toronto_venues.shape)
toronto_venues.head()

(1188, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.659343,-79.365525,Sukhothai,43.658444,-79.365681,Thai Restaurant
1,"Regent Park, Harbourfront",43.659343,-79.365525,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
2,"Regent Park, Harbourfront",43.659343,-79.365525,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
3,"Regent Park, Harbourfront",43.659343,-79.365525,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
4,"Regent Park, Harbourfront",43.659343,-79.365525,Paintbox Bistro,43.660050,-79.362855,Restaurant


In [159]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport",91,91,91,91,91,91
Church and Wellesley,89,89,89,89,89,89
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Downtown Toronto,56,56,56,56,56,56
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront, Union Station, Toronto Islands",91,91,91,91,91,91
"Kensington Market, Chinatown, Grange Park",100,100,100,100,100,100


In [160]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 192 uniques categories.


In [161]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [167]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

In [168]:
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Pool Hall,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rest Area,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0

In [169]:
toronto_onehot.shape

(1188, 192)

In [170]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Pool Hall,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rest Area,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint
0,Bay Street,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.010000,0.01,0.000000,0.01,0.000000,0.000000,0.020000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.020000,0.00,0.020000,0.010000,0.000000,0.040000,0.000000,0.000000,0.020000,0.00,0.090000,0.000000,0.080000,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.01,0.010000,0.000000,0.010000,0.000000,0.00,0.000000,0.01,0.000000,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.010000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.01,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.030000,0.020000,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.01,0.01,0.01,0.00,0.010000,0.010000,0.000000,0.010000,0.000000,0.000000,0.00,0.01,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.030000,0.000000,0.000000,0.00,0.000000,0.010000,0.01,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.01,0.010000,0.000000,0.000000,0.

In [171]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Street----
            venue  freq
0  Clothing Store  0.09
1     Coffee Shop  0.08
2            Café  0.04
3           Plaza  0.03
4  Cosmetics Shop  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport----
                venue  freq
0               Hotel  0.10
1         Coffee Shop  0.09
2         Pizza Place  0.04
3          Sports Bar  0.04
4  Italian Restaurant  0.04


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.07
1  Japanese Restaurant  0.06
2         Burger Joint  0.04
3     Sushi Restaurant  0.04
4              Gay Bar  0.04


----Commerce Court, Victoria Hotel----
                 venue  freq
0          Coffee Shop  0.11
1                Hotel  0.06
2                 Café  0.05
3  American Restaurant  0.04
4           Restaurant  0.04


----Downtown Toronto----
            venue  freq
0    Cocktail Bar  0.05
1     Coffee Shop  0.05
2     Cheese Shop  0.04
3      Steakhouse  0.04
4  Fa

In [172]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [173]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Street,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Plaza,Italian Restaurant,Tea Room,Restaurant,Sandwich Place,Burger Joint
1,"CN Tower, King and Spadina, Railway Lands, Har...",Hotel,Coffee Shop,Pizza Place,Italian Restaurant,Sports Bar,Aquarium,Gym,Bar,Scenic Lookout,Bistro
2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Burger Joint,Gay Bar,Restaurant,Men's Store,Café,Fast Food Restaurant,Bubble Tea Shop
3,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Italian Restaurant,Seafood Restaurant,Gastropub,Japanese Restaurant,Cocktail Bar
4,Downtown Toronto,Coffee Shop,Cocktail Bar,Farmers Market,Restaurant,Beer Bar,Café,Cheese Shop,Steakhouse,Seafood Restaurant,Bakery
5,"First Canadian Place, Underground city",Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Asian Restaurant,Gastropub,Burger Joint,Deli / Bodega,Steakhouse
6,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Burger Joint,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Sandwich Place,Tea Room,Pizza Place
7,"Harbourfront, Union Station, Toronto Islands",Coffee Shop,Boat or Ferry,Pizza Place,Scenic Lookout,Café,Brewery,Gym,Aquarium,Restaurant,Baseball Stadium
8,"Kensington Market, Chinatown, Grange Park",Chinese Restaurant,Bar,Vietnamese Restaurant,Café,Yoga Studio,Arts & Crafts Store,Coffee Shop,French Restaurant,Art Gallery,Cocktail Bar
9,North Toronto,Sporting Goods Shop,Coffee Shop,Yoga Studio,Clothing Store,Spa,Mexican Restaurant,Sandwich Place,Salon / Barbershop,Café,Chinese Restaurant


In [174]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 4], dtype=int32)

In [175]:
toronto_merged = toronto_df

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.659343,-79.365525,3,Café,Restaurant,Pharmacy,Grocery Store,Yoga Studio,Chinese Restaurant,Convenience Store,Mexican Restaurant,Event Space,Sandwich Place
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657658,-79.378802,3,Coffee Shop,Clothing Store,Café,Burger Joint,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Sandwich Place,Tea Room,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.668195,-79.373771,3,Pizza Place,Coffee Shop,Restaurant,Indian Restaurant,Café,Plaza,Butcher,Steakhouse,Beer Store,Market
3,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306,3,Coffee Shop,Cocktail Bar,Farmers Market,Restaurant,Beer Bar,Café,Cheese Shop,Steakhouse,Seafood Restaurant,Bakery
4,M5G,Downtown Toronto,Bay Street,43.655854,-79.383778,3,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Plaza,Italian Restaurant,Tea Room,Restaurant,Sandwich Place,Burger Joint


Finally, Visualizing the resulting clusters

In [177]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters